In [5]:
import pygame
import numpy as np

class Cubinho:
    def __init__(self, coordenada, adesivos, tipo_de_peça):
        self.coordenada = coordenada
        self.adesivos = adesivos
        self.tipo_de_peça = tipo_de_peça

class Cubo:
    def __init__(self):
        self.estado_inicial = []
        self.lista_de_cubinhos = []
        self.coordenadas = []
        for z in range(-1, 2):
            for y in range(-1, 2):
                for x in range(-1, 2):
                    self.coordenadas.append(np.array([x, y, z]))

        for item in self.coordenadas:
            tipo_de_peça = None
            adesivos = {}
            x, y, z = item

            if x == 1:
                adesivos['azul'] = np.array([1, 0, 0])
            if x == -1:
                adesivos['verde'] = np.array([-1, 0, 0])
            if y == 1:
                adesivos['vermelho'] = np.array([0, 1, 0])
            if y == -1:
                adesivos['laranja'] = np.array([0, -1, 0])
            if z == 1:
                adesivos['amarelo'] = np.array([0, 0, 1])
            if z == -1:
                adesivos['branco'] = np.array([0, 0, -1])

            if len(adesivos) == 2:
                tipo_de_peça = 'meio'
            if len(adesivos) == 3:
                tipo_de_peça = 'quina'
            if len(adesivos) == 1:
                tipo_de_peça = 'centro'
            if len(adesivos) == 0:
                tipo_de_peça = 'nucleo'

            self.lista_de_cubinhos.append(Cubinho(item, adesivos, tipo_de_peça))
            self.estado_inicial.append(Cubinho(item, adesivos, tipo_de_peça))

    def rotacionar(self, angulo_x, angulo_y, angulo_z):
        """Rotaciona todos os cubinhos em relação à origem."""
        for cubinho in self.lista_de_cubinhos:
            x, y, z = cubinho.coordenada
            
            # Rotação em torno do eixo Z
            x_rot_z = x * np.cos(angulo_z) - y * np.sin(angulo_z)
            y_rot_z = x * np.sin(angulo_z) + y * np.cos(angulo_z)
            z_rot_z = z

            # Rotação em torno do eixo Y
            x_rot_y = x_rot_z * np.cos(angulo_y) + z_rot_z * np.sin(angulo_y)
            z_rot_y = -x_rot_z * np.sin(angulo_y) + z_rot_z * np.cos(angulo_y)
            y_rot_y = y_rot_z

            # Rotação em torno do eixo X
            x_rot_x = x_rot_y
            y_rot_x = y_rot_y * np.cos(angulo_x) - z_rot_y * np.sin(angulo_x)
            z_rot_x = y_rot_y * np.sin(angulo_x) + z_rot_y * np.cos(angulo_x)

            cubinho.coordenada = np.array([x_rot_x, y_rot_x, z_rot_x])

    def projetar(self):
        """Projeta as coordenadas 3D para 2D."""
        projetados = []
        for cubinho in self.lista_de_cubinhos:
            x, y, z = cubinho.coordenada

            # Projeção 2D
            fator_escala = 100
            proj_x = int(x * fator_escala) + 400  # Centraliza a tela
            proj_y = int(-y * fator_escala) + 300  # Inverte Y para Pygame
            projetados.append((proj_x, proj_y, cubinho.adesivos))
        return projetados

def desenhar_cubos(screen, cubos):
    """Desenha os cubos na tela com base nas coordenadas projetadas."""
    screen.fill((255, 255, 255))  # Limpa a tela

    projetados = cubos.projetar()
    for (x, y, adesivos) in projetados:
        # Desenha apenas as faces externas
        for cor, pos in adesivos.items():
            if pos[0] == 1:  # Azul
                pygame.draw.rect(screen, (0, 0, 255), (x, y, 50, 50))
            elif pos[0] == -1:  # Verde
                pygame.draw.rect(screen, (0, 255, 0), (x, y, 50, 50))
            elif pos[1] == 1:  # Vermelho
                pygame.draw.rect(screen, (255, 0, 0), (x, y, 50, 50))
            elif pos[1] == -1:  # Laranja
                pygame.draw.rect(screen, (255, 165, 0), (x, y, 50, 50))
            elif pos[2] == 1:  # Amarelo
                pygame.draw.rect(screen, (255, 255, 0), (x, y, 50, 50))
            elif pos[2] == -1:  # Branco
                pygame.draw.rect(screen, (255, 255, 255), (x, y, 50, 50))

    pygame.display.flip()  # Atualiza a tela

def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    pygame.display.set_caption('Cubo Mágico 3D')
    clock = pygame.time.Clock()

    cubos = Cubo()
    angulo_x, angulo_y, angulo_z = 0, 0, 0
    rodando = True

    while rodando:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                rodando = False

        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            angulo_y -= 0.05
        if keys[pygame.K_RIGHT]:
            angulo_y += 0.05
        if keys[pygame.K_UP]:
            angulo_x -= 0.05
        if keys[pygame.K_DOWN]:
            angulo_x += 0.05
        if keys[pygame.K_a]:  # Rotaciona em torno do eixo Z no sentido anti-horário
            angulo_z -= 0.05
        if keys[pygame.K_d]:  # Rotaciona em torno do eixo Z no sentido horário
            angulo_z += 0.05

        cubos.rotacionar(angulo_x, angulo_y, angulo_z)  # Aplica a rotação
        desenhar_cubos(screen, cubos)
        clock.tick(60)

    pygame.quit()

if __name__ == "__main__":
    main()
